# Coastal Model Test Bed (CMTB) SWASH 1D Example 

#### This notebook walks through the CMTB SWASH 1D Workflow
    CMTB sets this up as a "skinny" 2D simulation that is effectively 1D
    Notebook users will have to download CMTB from github and install the required python packages
    Users select simulation settings in the User Input Cell and can walk through the rest of the code
    Once CMTB writes the input files, users can run SWASH separately in a command line interface 
    (commands listed at end of notebook) OR, if users have a SWASH mpiexec compiled (aka SWASH compiled in parallel mode),    
    users may run SWASH via CMTB

## Load packages and libraries

In [ ]:
import sys
sys.path.insert(0, '/Users/lszcz/Documents/CMTB/cmtb/')

# CMTB 
from prepdata.prepDataLib import PrepDataTools as preptools
from getdatatestbed import getDataFRF
from getdatatestbed.getDataFRF import getDataTestBed
from prepdata import writeRunRead as wrr
from prepdata import inputOutput
from testbedutils import waveLib as sbwave
from subprocess import check_output
from plotting.operationalPlots import generate_CrossShoreTimeseries
from plotting import operationalPlots as oP
from testbedutils import sblib as sb

# Standard utilities
import datetime as dt
import netCDF4 as nc
import numpy as np
import os, glob, makenc
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc

## User Input Cell: Declare input settings

In [ ]:
# Define various directories:
path_prefix = '/Users/lszcz/Documents/CMTB/cmtb'         # core cmtb path
workingDir = '/Users/lszcz/Documents/CMTB/cmtb/data'   
exe = '/mirror/swash/swash.exe'
# choose location and name of netcdf file with results:
npath= '/Users/lszcz/Documents/CMTB/cmtb/data/swash1D_test_mpi/ncresults/results.nc'    
# local path to swash_global.yml:
fieldYaml = '/Users/lszcz/Documents/CMTB/cmtb/yaml_files/WaveModels/swash/base/swash_global.yml'  
# local path to swash_var.yml:
varYaml = '/Users/lszcz/Documents/CMTB/cmtb/yaml_files/waveModels/swash/swash_var.yml' 

# Define simulation
testName = 'swash1D_test_mpi'
versionPrefix='base'
model = 'swash'

# Select simulated period
date_str = '2019-10-11' 
startTime = dt.datetime(2019,10,11,15,0,0)          # Start time, Year, month, day, hour
endTime = dt.datetime(2019,10,11,16,0,0)            # Stop time, 1 hour after start time, Year, month, day, hour
simulationDuration = 1                              # length of time for simulated event (should be 1 hour)
spinup = 900                                        # initial spinup time before model output (in seconds)
ncores = 18                                         # number of cores to use

# Select bathymetric setting
bathy_loc = 'survey'
profile_num = 945                                   # FRF profile number
survey_date = dt.datetime(2019, 10, 15, 23)         # Select survey date (always include 23 aka 11 pm to search for any profile collected that day)
xmin = 25                                           # Onshore extent in FRF coordinates
xmax = 915                                          # Offshore extent in FRF coordinates
dx = 1                                              # Resolution in x
dy = 1                                              # Resolution in y 
fric_fac = 0.015

# Select sensor for boundary conditions
sensor = '8m-array'                                 # FRF Sensor used to pull wave conditions

***
***
***

## Auto-format additional variables

In [ ]:
runDuration = (endTime - startTime).total_seconds()
ymin = profile_num - dy                                              # Extending selected transect for "Psuedo 1D" simulation, allowing for a skinny alongshore interpolated grid
ymax = profile_num + dy

waveTimeList = preptools.timeLists(startTime,endTime,30*60)          # dt in hours
wlTimeList = preptools.timeLists(startTime+dt.timedelta(minutes=30), # Finds 1 water level halfway through the target event
                                 endTime,30*60)  

## Gather raw data

Waves and water levels

In [ ]:
go = getDataFRF.getObs(startTime,endTime)    # initialize go command with simulated period

In [ ]:
wave_spec = go.getWaveData(sensor,spec=True) # grab raw spectra
wl = go.getWL()                              # grab raw water levels

Bathymetry

In [ ]:
gdTB = getDataTestBed(startTime, endTime)    # initialize gdTB command with simulated period

In [ ]:
bathy_data = gdTB.getBathyIntegratedTransect(method=1,                     # Gathers relevant data (beyond designated bounds) to pass into prepbathy for interpolation
                                             ForcedSurveyDate=survey_date, 
                                             ybounds=[ymin, ymax])        

Check out raw data

In [ ]:
plt.figure()
plt.plot(wave_spec['wavefreqbin'], wave_spec['fspec'])
plt.ylabel('E [$m^2/Hz$]')
plt.xlabel('Frequency [$Hz$]')
plt.show()

In [ ]:
rmesh, thetamesh = np.meshgrid(wave_spec['wavefreqbin'],wave_spec['wavedirbin'])
thetamesh = thetamesh*np.pi/180
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
E_2D_plot = wave_spec['dWED'].T
ax.contourf(np.flip(thetamesh), rmesh,E_2D_plot,10)
ax.set_theta_zero_location("N")
plt.show()

In [ ]:
plt.plot(wl['time'], wl['WL'])
plt.ylabel('$\eta$ [$m$]')
plt.xlabel('time [$s$]')
plt.show()

## Format gathered raw data for SWASH input

In [ ]:
prepdata = preptools() # initialize prepdata command

Process wave packet

In [ ]:
good_freqs = wave_spec['wavefreqbin'][wave_spec['fspec'].mask == False]   
    # Above needed otherwise will interpolated frequencies beyond sensor range, skewing energy levels down to -999
wavePacket = prepdata.prep_spec_phaseResolved(rawspec=wave_spec, 
                                              version_prefix=versionPrefix, 
                                              grid='1D',
                                              spinUpTime=spinup, 
                                              runDuration=runDuration,
                                              freqRange=[min(good_freqs), max(good_freqs)]) # Filter down to "good" frequencies

Process water level packet

In [ ]:
wlPacket = prepdata.prep_WL(wl,wlTimeList)
        # Should return 1 record for SWASH input

Process bathy packet

In [ ]:
bathyPacket = prepdata.prep_SwashBathy(x0=xmin, 
                                       y0=profile_num-1, 
                                       bathy=bathy_data, 
                                       xBounds=[xmin, xmax+1], 
                                       yBounds = [ymin,ymax+1], 
                                       dx=dx, dy=dy)
        # if 1D, ybounds should be numbers surrounding target profile number (established in user input cell)
        # Xbounds and Ybounds plus 1 because python is 0-indexed

Check out processed data

In [ ]:
h = bathyPacket['elevation']
h = h[2,:]                          # Identify target profile
plt.figure()
plt.plot(bathyPacket['xFRF'], h)
plt.ylabel('h [$m$]')
plt.xlabel('x [$m$]')
plt.show()

In [ ]:
plt.figure()
plt.plot(wavePacket['freqbins'], wavePacket['fspec'])
plt.ylabel('E [$m^2/Hz$]')
plt.xlabel('Frequency [$Hz$]')
plt.show()

In [ ]:
plt.figure()
plt.plot(wl['time'],wl['WL'])
plt.plot(wlPacket['time'],wlPacket['avgWL'],marker="o", markersize=20)
plt.ylabel('Water Level (m)')
plt.xlabel('Date')
plt.legend(('Raw WL','Input WL'))
plt.show()

## Setup the SWASH model

In [ ]:
swio = wrr.swashIO(WL=wlPacket['avgWL'], 
                   equilbTime=wavePacket['spinUp'],  
                   Hs=wavePacket['Hs'], 
                   Tp=1/wavePacket['peakf'],
                   Dm=wavePacket['waveDm'],
                   workingDirectory=workingDir, 
                   testName=testName, fileNameBase=date_str,
                   version_prefix=versionPrefix, 
                   runTime=simulationDuration, 
                   startTime=startTime, 
                   endTime=endTime,
                   runFlag=True, 
                   generateFlag=False,
                   readFlag=True)
    # Above initializes swio command

In [ ]:
swio._replaceDefaultParams()                    # fill out swio command with default model settings

if not os.path.isdir(swio.workingDirectory):    # make directory where input files stored
    os.makedirs(swio.workingDirectory)
os.chdir(swio.workingDirectory)

swio._check_input()                             # final checks before writing

Write the input files

In [ ]:
swio.write_spec1D(wavePacket['freqbins'], wavePacket['fspec'])      # writes the SWASH forcing.BND file
swio.write_bot(bathyPacket['elevation'])                            # writes the SWASH bathy.BOT file
swio.write_sws(gridDict=bathyPacket, WLpacket=wlPacket)             # writes the SWASH INPUT.sws file

##  -----------------------------------------------------------------------------------------
# RUNING SWASH BELOW, TWO OPTIONS:
## (1) Run SWASH separately 
    Switch to command line, cd to the test directory with input files, and run 'swash INPUT'

## (2) User runs SWASH mpiexec via this notebook in the following cells: 
    Can be applied if user has SWASH compiled in parallel mode on their machine and can call mpiexec:

In [ ]:
swio._generateRunStrings(exe) # default run string generated by cmtb: 'mpiexec -n 18 /USER/PATHTO.EXE INPUT'
# overwriting below to account for windows running ubuntu shell on windows machine
swio.runString1 = 'wsl mpiexec -n 18 /mirror/swash/swash.exe INPUT' 

In [ ]:
swio._preprocessCheck()
init_t = dt.datetime.now()
print('Running {} Simulation starting at {}'.format(swio.modelName, init_t))

_ = check_output(swio.runString1, shell=False)            # RUN COMMAND

## -----------------------------------------------------------------------------------------
## Load in results

In [ ]:
os.chdir(swio.workingDirectory)                               # switch to where .mat results stored (working directory)
[dataDict, metaDict] = swio.loadSwash_Mat(testName+'.mat')    # load in and format results
    # (produces a benign warning)

Calculate some wave statistics

In [ ]:
eta = dataDict['eta'].squeeze()

fspec, freqs = sbwave.timeSeriesAnalysis1D(dataDict['time'].squeeze(), dataDict['eta'].squeeze(), bandAvg=6)
Stats = sbwave.stats1D(fspec=fspec, frqbins=freqs, lowFreq=None, highFreq=None)
    # (produces a benign warning)
HsTS = 4 * np.std(dataDict['eta'].squeeze(), axis=0)

In [ ]:
# We use 0.08 m for runup threshold
r_depth = 0.08  # 4.0 * np.nanmax(np.abs(h[runupInd][1:] - h[runupInd][:-1]))

# Pre-allocate runup variable
runup = np.zeros(eta.shape[0])
x_runup = np.zeros_like(runup)

for aa in range(runup.shape[0]):
        # Water depth
        wdepth = eta[aa, :] + dataDict['elevation']
        # Find the runup contour (search from left to right)
        wdepth_ind = np.argmin(abs(wdepth - r_depth))  # changed from Chuan's original code
        # Store the water surface elevation in matrix
        runup[aa] = eta[aa, wdepth_ind]  # unrealistic values for large r_depth
        # runup[aa]= -h[wdepth_ind]
        # Store runup position
        x_runup[aa] = dataDict['xFRF'][wdepth_ind]
maxRunup = np.amax(runup)

## Check out processed results

Plot eta

In [ ]:
plt.figure()
plt.plot(dataDict['time'],dataDict['eta'][:,0,400])
plt.ylabel('$\eta$ (m)')
plt.xlabel('Date')
plt.show()

Plot Hs

In [ ]:
plt.figure()
plt.plot(dataDict['xFRF'],HsTS)
plt.ylabel('$H_s$ (m)')
plt.xlabel('x (m)')
plt.show()

Plot setup

In [ ]:
if not os.path.exists(os.path.join(path_prefix, 'figures')):
        os.makedirs(os.path.join(path_prefix, 'figures'))  # make a directory 'figures' for the simulation plots
figurepath = workingDir+'/'+testName+'/'+'figures'

In [ ]:
generate_CrossShoreTimeseries(figurepath+'/setup', np.mean(dataDict['eta'].squeeze(), axis=0), dataDict['elevation'], dataDict['xFRF'])

In [ ]:
from IPython.display import Image
Image(figurepath+'/setup'+'.png')

Plot timeseries of eta

In [ ]:
figureBaseFname = 'CMTB_waveModels_{}_{}_'.format(model, versionPrefix) # names the images generated

In [ ]:
# choose the subsample of timesteps to include in the gif
nSubSample = 2
# Choose frames per second of the generated gif ( fps = nSubSample*dt -->> default option)
fps = 20

In [ ]:
for tidx in np.arange(0, len(dataDict['time']), nSubSample).astype(int):
            ofPlotName = os.path.join(figurepath, figureBaseFname + 'TS_' + dataDict['time'][tidx].strftime('%Y%m%dT%H%M%S%fZ') +'.png')
            oP.generate_CrossShoreTimeseries(ofPlotName, dataDict['eta'][tidx].squeeze(), -dataDict['elevation'], dataDict['xFRF'])

In [ ]:
imgList = sorted(glob.glob(os.path.join(figurepath, '*_TS_*.png')))
dt = np.median(np.diff(dataDict['time'])).microseconds / 1000000
sb.makeMovie(os.path.join(figurepath, figureBaseFname + 'TS_{}.avi'.format(date_str)), imgList, fps=fps)
tarOutFile = os.path.join(figurepath, figureBaseFname + 'TS.tar.gz')
sb.myTarMaker(tarOutFile, imgList)

In [ ]:
from IPython.display import Video
Video(figurepath+'/'+figureBaseFname + 'TS_{}.mp4'.format(date_str), embed=True)

## Save Results to netcdf

Format a data dictionary for proper netcdf storage

In [ ]:
deet = np.median(np.diff(dataDict['time'])).microseconds / 1000000
tsTime = np.arange(0, len(dataDict['time'])*deet, deet)
spatial = {'time': nc.date2num(startTime, units='seconds since 1970-01-01 00:00:00'),
               'station_name': '{} Field Data'.format(model),
               'tsTime': tsTime,
               'waveHsIG': np.reshape(Stats['Hm0'], (1, len(dataDict['xFRF']))),
               'eta': np.swapaxes(dataDict['eta'], 0, 1),
               'totalWaterLevel': maxRunup,
               'totalWaterLevelTS': np.reshape(runup, (1, len(runup))),
               'velocityU': np.swapaxes(dataDict['velocityU'], 0, 1),
               'velocityV': np.swapaxes(dataDict['velocityV'], 0, 1),
               'waveHs': np.reshape(Stats['Hm0'], (1, len(dataDict['xFRF']))), # or from HsTS??
               'xFRF': dataDict['xFRF'],
               'yFRF': dataDict['yFRF'][0],
               'runTime': np.expand_dims(swio.simulationWallTime, axis=0),
               'DX': np.median(np.diff(dataDict['xFRF'])).astype(int),
               'DY': 1,    # must be adjusted for 2D simulations
               'NI': len(dataDict['xFRF']),
               'NJ': dataDict['velocityU'].shape[1],}  # should automatically adjust for 2D simulations

In [ ]:
flagfname = os.path.join(npath, 'Flags{}.out.txt'.format(date_str)) # the name of flag file
makenc.makenc_phaseresolved(data_lib=spatial, globalyaml_fname=fieldYaml, ofname=npath, flagfname=flagfname,
                            var_yaml_fname=varYaml)